In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import openpyxl
from openpyxl import Workbook
import networkx as nx
from openpyxl import load_workbook
import matplotlib.pyplot as plt
from openpyxl.utils import get_column_letter
from random import randint
from mysql.connector import connect
import networkx as nx
import pandas as pd
import community as community_louvain

In [2]:
data = pd.read_csv("./holder_edge/result.csv",names=['from_addr','to_addr'],sep="|")
data.head()

,from_addr,to_addr
0,0xafaa71efde96621839214ed8f763f7eb8936be0a,0xb12f8f13ba8e5064c4dd63cee507fd5fa2b7782c
1,0xbf5928c1d26d956805bbbfb6986b445b737618dd,0x9ffae5caa64c33549122e5a68140abe0c4192a9a
2,0x2a613275bb355407bd90ea3e4e5ed0fcdb7107e8,0xd77579390405d1e2a106f777104022a950c5d7c9
3,0x625f880282786d90769a657d5cc2e3b34c4e2147,0x5da2690b20cdafb31eee5ccacfd59009326e9e28
4,0xa75fee88e663bcd2f8ebd8203877cb963d9d37db,0x85c484dbce607ab15d4be2b24373b3836e9c6437


In [3]:

# 删除热点
def select_hot_address(data, field1, field2, n):
    from_to_cnt = data.groupby([field1]).agg({field2:'nunique'}).add_suffix("_cnt")
    source_from_to_cnt = from_to_cnt.sort_values(by=[field2+'_cnt'], ascending=False)
    hot_address_list = source_from_to_cnt.index.tolist()[:n]
    return hot_address_list

# 图算法，先用连通图，然后剔热点，然后用louvain算法
def recursive_louvain(graph, cluster_size):# 参数分别为进入的df和允许的最大cluster的节点数
    sub_graph_edge = []
    node_info = pd.DataFrame()
    g = nx.Graph()
    g.add_edges_from(graph[['from_addr','to_addr']].values)
# 连通图算法
    for index,sub_g in enumerate(nx.connected_components(g)):
        sub_g = g.subgraph(sub_g)
        g_node = sub_g.nodes(sub_g)
        g_node = [g_per[0] for g_per in g_node]
        g_node_tag = [index]*len(g_node)
        node_tag_df = pd.DataFrame()
        node_tag_df['node_addr'] = g_node
        node_tag_df['cluster_id'] = g_node_tag
        node_info = pd.concat([node_info, node_tag_df])

        edg_df = pd.DataFrame(list(sub_g.edges()))
        edg_df.columns = ['from_addr','to_addr']
        edg_df['node_cnt'] = len(sub_g.nodes())
        edg_df['tag'] = index
        sub_graph_edge.append(edg_df)

    sub_graph_edge = pd.concat(sub_graph_edge,ignore_index=True)# 存储连通子图
    node_infos_size=node_info.groupby('cluster_id').size().reset_index() # 对结果计算cluster的size
    node_infos_size.columns=['cluster_id','size']
    large_clusters=node_infos_size[node_infos_size['size']>cluster_size]['cluster_id'].to_list()# 筛选大于最大size的数据
    small_clusters=node_infos_size[node_infos_size['size']<=cluster_size]['cluster_id'].to_list()# 筛选小于等于最大size的数据
    samll_node_info_df=node_info[node_info['cluster_id'].isin(small_clusters)]# 将小于等于最大size的编号对应关系保存
    if len(large_clusters)==0:
        result_df= samll_node_info_df
        return result_df
    else:
        result_df = pd.DataFrame()
        for c in large_clusters:# 针对大于最大size的进行下一轮算法
            sug_graph = sub_graph_edge[sub_graph_edge['tag'] == c]
            # 剔除热点
            from_hot_addr_list = select_hot_address(sug_graph, 'from_addr', 'to_addr', 1)
            to_hot_addr_list = select_hot_address(sug_graph, 'to_addr', 'from_addr', 1)
            del_hot_addr_df = sug_graph[~((sug_graph['from_addr'].isin(from_hot_addr_list))|(sug_graph['to_addr'].isin(to_hot_addr_list)))]
            #用louvain算法
            G = nx.Graph()
            G.add_edges_from(del_hot_addr_df[['from_addr', 'to_addr']].values)
            partition = community_louvain.best_partition(G, resolution=0.1)
            node_info_cluster = pd.DataFrame(list(partition.items()), columns=['node_addr', 'cluster_id'])
            # 对新一轮计算出来的cluster_id做上标记，防止与前面编号冲突
            node_info_cluster['cluster_id'] = node_info_cluster['cluster_id'].apply(lambda x: str(x) + "_" + str(c))
            result_df = pd.concat([result_df, node_info_cluster])
        # 存储前面小于等于最大size的数据和后续被重新划分后的数据
        result_df = pd.concat([samll_node_info_df, result_df])
    return result_df

In [4]:
result_df = recursive_louvain(data, 200)

In [5]:
result_df.to_csv("cluster_result_v1.csv",index=False)
'done'

'done'

In [7]:
result_df = pd.read_csv("cluster_result.csv")
result_df.head()

,node_addr,cluster_id
0,0x3506fbe85e19bf025b228ec58f143ba342c3c608,0
1,0x00d86cf36cfa992e69dcb51f0429eea74f105791,0
2,0x0a6a81bfeb4d7a58f61ba9f7b22ef3fc69bf8488,0
3,0xafaa71efde96621839214ed8f763f7eb8936be0a,0
4,0xd89ce37f247290f4b404db7415deed1a0cf163b1,0


In [13]:
result_df

,node_addr,cluster_id
0,0x3506fbe85e19bf025b228ec58f143ba342c3c608,0
1,0x00d86cf36cfa992e69dcb51f0429eea74f105791,0
2,0x0a6a81bfeb4d7a58f61ba9f7b22ef3fc69bf8488,0
3,0xafaa71efde96621839214ed8f763f7eb8936be0a,0
4,0xd89ce37f247290f4b404db7415deed1a0cf163b1,0
...,...,...
184,0x38ac15bf7eea4468d51348f011e9d89ab2d21479,14_588
185,0x1332372b89a64cf1b7f0ad56ee15ec78d633cc39,61_588
186,0x94bb227d3834978ed32e3b166707bc09a258d6d4,61_588
0,0x038d7bbc67055fa614144db8d2519c4a02bc6aea,0_2772


In [8]:
data_new_group = pd.read_csv("data_new_hour_gp.csv")
data_new_group.head()

,from_address,to_address,function_name,block_time,rank1,from_address2,address_cnt,group_id,time1,rank,gap_time,group_id_hour_0.5
0,0xe1d01a1a6b3b5f04ebc8cc8242e0a3b9488b59ff,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 22:27:52,1,0xe1d01a1a6b3b5f04ebc8cc8242e0a3b9488b59ff,45486,0,2022-05-18 22:33:48,1.0,5.933333,7879
1,0xdde199991ad1a917be29b92618dbb1daa464372e,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 22:33:48,1,0xdde199991ad1a917be29b92618dbb1daa464372e,45486,0,2022-05-18 22:41:16,2.0,7.466667,7879
2,0xa6ce862bad1f4fe36c6297f33aada06c0d39dfa3,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 22:41:16,1,0xa6ce862bad1f4fe36c6297f33aada06c0d39dfa3,45486,0,2022-05-18 22:43:04,3.0,1.800000,7879
3,0xe8265d372266f45f99fd4e13fbc0f1d7d59a12ea,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 22:43:04,1,0xe8265d372266f45f99fd4e13fbc0f1d7d59a12ea,45486,0,2022-05-18 22:43:26,4.0,0.366667,7879
4,0xf3e1b1cba22e60958081f2d2b3d5a336e5462adc,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 22:43:26,1,0xf3e1b1cba22e60958081f2d2b3d5a336e5462adc,45486,0,2022-05-18 22:44:10,5.0,0.733333,7879


In [9]:
data_new_group[data_new_group['function_name'] == '0xa22cb465']

,from_address,to_address,function_name,block_time,rank1,from_address2,address_cnt,group_id,time1,rank,gap_time,group_id_hour_0.5
0,0xe1d01a1a6b3b5f04ebc8cc8242e0a3b9488b59ff,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 22:27:52,1,0xe1d01a1a6b3b5f04ebc8cc8242e0a3b9488b59ff,45486,0,2022-05-18 22:33:48,1.0,5.933333,7879
1,0xdde199991ad1a917be29b92618dbb1daa464372e,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 22:33:48,1,0xdde199991ad1a917be29b92618dbb1daa464372e,45486,0,2022-05-18 22:41:16,2.0,7.466667,7879
2,0xa6ce862bad1f4fe36c6297f33aada06c0d39dfa3,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 22:41:16,1,0xa6ce862bad1f4fe36c6297f33aada06c0d39dfa3,45486,0,2022-05-18 22:43:04,3.0,1.800000,7879
3,0xe8265d372266f45f99fd4e13fbc0f1d7d59a12ea,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 22:43:04,1,0xe8265d372266f45f99fd4e13fbc0f1d7d59a12ea,45486,0,2022-05-18 22:43:26,4.0,0.366667,7879
4,0xf3e1b1cba22e60958081f2d2b3d5a336e5462adc,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 22:43:26,1,0xf3e1b1cba22e60958081f2d2b3d5a336e5462adc,45486,0,2022-05-18 22:44:10,5.0,0.733333,7879
...,...,...,...,...,...,...,...,...,...,...,...,...
45481,0x55f31ba9ff2224449430f1eb1a924b8c291f89d2,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2023-04-05 00:39:08,1,0x55f31ba9ff2224449430f1eb1a924b8c291f89d2,45486,0,2023-04-05 01:35:34,44340.0,56.433333,10556
45482,0x52de77d5443fe553cd0fa24b4a0f0d42ab9fd3a8,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2023-04-05 01:35:34,1,0x52de77d5443fe553cd0fa24b4a0f0d42ab9fd3a8,45486,0,2023-04-05 01:55:52,44341.0,20.300000,1102
45483,0x98ac33bce32abe56d22600ef1d6945df4d8d9c74,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2023-04-05 01:55:52,1,0x98ac33bce32abe56d22600ef1d6945df4d8d9c74,45486,0,2023-04-05 02:09:45,44342.0,13.883333,1102
45484,0xf7b3cc1b39d40cc1cbfeeaa11d51dc1356aad34c,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2023-04-05 02:09:45,1,0xf7b3cc1b39d40cc1cbfeeaa11d51dc1356aad34c,45486,0,2023-04-05 05:43:42,44343.0,213.950000,1102


In [10]:
data_new_group[data_new_group['function_name'] == '0xa22cb465']['from_address'].nunique()

45486

In [11]:
fun_data1 = data_new_group[data_new_group['function_name'] == '0xa22cb465']
result_df_new = result_df.rename(columns={'node_addr':'from_address'})
fun_same_action = fun_data1.merge(result_df_new,how='inner',on=['from_address'])
fun_same_action

,from_address,to_address,function_name,block_time,rank1,from_address2,address_cnt,group_id,time1,rank,gap_time,group_id_hour_0.5,cluster_id
0,0xe1d01a1a6b3b5f04ebc8cc8242e0a3b9488b59ff,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 22:27:52,1,0xe1d01a1a6b3b5f04ebc8cc8242e0a3b9488b59ff,45486,0,2022-05-18 22:33:48,1.0,5.933333,7879,604
1,0xdde199991ad1a917be29b92618dbb1daa464372e,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 22:33:48,1,0xdde199991ad1a917be29b92618dbb1daa464372e,45486,0,2022-05-18 22:41:16,2.0,7.466667,7879,7160
2,0xa6ce862bad1f4fe36c6297f33aada06c0d39dfa3,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 22:41:16,1,0xa6ce862bad1f4fe36c6297f33aada06c0d39dfa3,45486,0,2022-05-18 22:43:04,3.0,1.800000,7879,861
3,0xe8265d372266f45f99fd4e13fbc0f1d7d59a12ea,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 22:43:04,1,0xe8265d372266f45f99fd4e13fbc0f1d7d59a12ea,45486,0,2022-05-18 22:43:26,4.0,0.366667,7879,861
4,0xc4b1154cf8f93ada51d59e18a5ee046cb7dd3c1a,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 22:44:10,1,0xc4b1154cf8f93ada51d59e18a5ee046cb7dd3c1a,45486,0,2022-05-18 22:46:24,6.0,2.233333,7879,565_21
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30378,0x6ca8d97c0e400d32635c1c2e39ec63c2f7f69449,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2023-04-05 00:23:34,1,0x6ca8d97c0e400d32635c1c2e39ec63c2f7f69449,45486,0,2023-04-05 00:33:30,44338.0,9.933333,10556,9056
30379,0x52de77d5443fe553cd0fa24b4a0f0d42ab9fd3a8,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2023-04-05 01:35:34,1,0x52de77d5443fe553cd0fa24b4a0f0d42ab9fd3a8,45486,0,2023-04-05 01:55:52,44341.0,20.300000,1102,4511
30380,0x98ac33bce32abe56d22600ef1d6945df4d8d9c74,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2023-04-05 01:55:52,1,0x98ac33bce32abe56d22600ef1d6945df4d8d9c74,45486,0,2023-04-05 02:09:45,44342.0,13.883333,1102,670
30381,0xf7b3cc1b39d40cc1cbfeeaa11d51dc1356aad34c,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2023-04-05 02:09:45,1,0xf7b3cc1b39d40cc1cbfeeaa11d51dc1356aad34c,45486,0,2023-04-05 05:43:42,44343.0,213.950000,1102,304


In [19]:
fun_gp_cnt = fun_same_action.groupby(['group_id_hour_0.5','cluster_id']).agg({'from_address':'nunique'})
fun_gp_cnt = fun_gp_cnt.sort_values(by=['from_address'],ascending=False)
fun_gp_cnt = fun_gp_cnt[fun_gp_cnt['from_address'] > 5]
fun_gp_cnt = fun_gp_cnt.reset_index()
fun_same_action
fun_same_action_new = fun_same_action.merge(fun_gp_cnt[['group_id_hour_0.5','cluster_id']])
fun_same_action_new


,from_address,to_address,function_name,block_time,rank1,from_address2,address_cnt,group_id,time1,rank,gap_time,group_id_hour_0.5,cluster_id
0,0xa2231be122b2c7b03027daac6cf8a61e9488fadf,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 23:09:54,1,0xa2231be122b2c7b03027daac6cf8a61e9488fadf,45486,0,2022-05-18 23:12:44,27.0,2.833333,7879,1700
1,0x88ea0edbb8ad489a775334603fcb26cd4561498a,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 23:19:52,1,0x88ea0edbb8ad489a775334603fcb26cd4561498a,45486,0,2022-05-18 23:22:48,31.0,2.933333,7879,1700
2,0x45eeb12ba77d013e63b73b1c6d35e5cce6e7a24d,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 23:26:42,1,0x45eeb12ba77d013e63b73b1c6d35e5cce6e7a24d,45486,0,2022-05-18 23:27:02,34.0,0.333333,7879,1700
3,0x24860fe14b24cade5f02c1ee3a71ebba7a784105,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 23:27:02,1,0x24860fe14b24cade5f02c1ee3a71ebba7a784105,45486,0,2022-05-18 23:28:54,35.0,1.866667,7879,1700
4,0x4b5fd4bbfa23d9d00d1eb298e91d44612777f63c,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 23:28:54,1,0x4b5fd4bbfa23d9d00d1eb298e91d44612777f63c,45486,0,2022-05-18 23:32:58,36.0,4.066667,7879,1700
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8674,0xb8a3a99a4d7d4aeeee09204c590c50dd84b22aca,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2023-04-02 14:18:38,1,0xb8a3a99a4d7d4aeeee09204c590c50dd84b22aca,45486,0,2023-04-02 14:19:54,43989.0,1.266667,2054,927
8675,0x4806cbaf5ffa33098b85f0f39a2fc4d003fe57e5,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2023-04-02 14:19:54,1,0x4806cbaf5ffa33098b85f0f39a2fc4d003fe57e5,45486,0,2023-04-02 14:20:56,43990.0,1.033333,2054,927
8676,0x1f61f66b2e5cce264cf5c9b203cdaa27cb7ba3ac,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2023-04-02 14:20:56,1,0x1f61f66b2e5cce264cf5c9b203cdaa27cb7ba3ac,45486,0,2023-04-02 14:22:27,43991.0,1.516667,2054,927
8677,0xf4f310321211d926d606db734ad89c848ba48610,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2023-04-02 14:22:27,1,0xf4f310321211d926d606db734ad89c848ba48610,45486,0,2023-04-02 14:23:43,43992.0,1.266667,2054,927


In [21]:
fun_same_action_new['from_address'].nunique()

8679

In [23]:
fun_same_action_new.to_csv("fun_same_action_new.csv",index=False)
'done'

'done'

In [16]:
# 0x3506fbe85e19bf025b228ec58f143ba342c3c608
# fun_gp_cnt['from_address'].sum()
fun_same_action_new = fun_same_action.merge(fun_gp_cnt[['group_id_hour_0.5','cluster_id']],how='left',on=['group_id_hour_0.5','cluster_id'])
fun_same_action_new
# fun_same_action[(fun_same_action['group_id_hour_0.5'] == 5725) & (fun_same_action['cluster_id'] == 691)]

,from_address_x,to_address,function_name,block_time,rank1,from_address2,address_cnt,group_id,time1,rank,gap_time,group_id_hour_0.5,cluster_id,from_address_y
0,0xe1d01a1a6b3b5f04ebc8cc8242e0a3b9488b59ff,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 22:27:52,1,0xe1d01a1a6b3b5f04ebc8cc8242e0a3b9488b59ff,45486,0,2022-05-18 22:33:48,1.0,5.933333,7879,604,NaN
1,0xdde199991ad1a917be29b92618dbb1daa464372e,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 22:33:48,1,0xdde199991ad1a917be29b92618dbb1daa464372e,45486,0,2022-05-18 22:41:16,2.0,7.466667,7879,7160,NaN
2,0xa6ce862bad1f4fe36c6297f33aada06c0d39dfa3,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 22:41:16,1,0xa6ce862bad1f4fe36c6297f33aada06c0d39dfa3,45486,0,2022-05-18 22:43:04,3.0,1.800000,7879,861,NaN
3,0xe8265d372266f45f99fd4e13fbc0f1d7d59a12ea,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 22:43:04,1,0xe8265d372266f45f99fd4e13fbc0f1d7d59a12ea,45486,0,2022-05-18 22:43:26,4.0,0.366667,7879,861,NaN
4,0xc4b1154cf8f93ada51d59e18a5ee046cb7dd3c1a,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-18 22:44:10,1,0xc4b1154cf8f93ada51d59e18a5ee046cb7dd3c1a,45486,0,2022-05-18 22:46:24,6.0,2.233333,7879,565_21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30378,0x6ca8d97c0e400d32635c1c2e39ec63c2f7f69449,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2023-04-05 00:23:34,1,0x6ca8d97c0e400d32635c1c2e39ec63c2f7f69449,45486,0,2023-04-05 00:33:30,44338.0,9.933333,10556,9056,NaN
30379,0x52de77d5443fe553cd0fa24b4a0f0d42ab9fd3a8,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2023-04-05 01:35:34,1,0x52de77d5443fe553cd0fa24b4a0f0d42ab9fd3a8,45486,0,2023-04-05 01:55:52,44341.0,20.300000,1102,4511,NaN
30380,0x98ac33bce32abe56d22600ef1d6945df4d8d9c74,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2023-04-05 01:55:52,1,0x98ac33bce32abe56d22600ef1d6945df4d8d9c74,45486,0,2023-04-05 02:09:45,44342.0,13.883333,1102,670,NaN
30381,0xf7b3cc1b39d40cc1cbfeeaa11d51dc1356aad34c,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2023-04-05 02:09:45,1,0xf7b3cc1b39d40cc1cbfeeaa11d51dc1356aad34c,45486,0,2023-04-05 05:43:42,44343.0,213.950000,1102,304,NaN


In [27]:
fun_same_action[(fun_same_action['group_id_hour_0.5'] == 7879) & (fun_same_action['cluster_id'] == 1104)]

,from_address,to_address,function_name,block_time,rank1,from_address2,address_cnt,group_id,time1,rank,gap_time,group_id_hour_0.5,cluster_id
57,0x328fa3ca77169df0a48c08ef9d6df9ef9aae090b,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-19 01:27:32,1,0x328fa3ca77169df0a48c08ef9d6df9ef9aae090b,45486,0,2022-05-19 01:31:02,74.0,3.500000,7879,1104
64,0x73ed939dbc3d1c525075eb619b41b8767f866be1,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-19 01:43:38,1,0x73ed939dbc3d1c525075eb619b41b8767f866be1,45486,0,2022-05-19 01:44:02,82.0,0.400000,7879,1104
65,0x4b8f22a9f7b26ef74771a7790d7fcf7755fed3f2,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-19 01:45:06,1,0x4b8f22a9f7b26ef74771a7790d7fcf7755fed3f2,45486,0,2022-05-19 01:45:44,84.0,0.633333,7879,1104
66,0x464d47e357e52d40a9f5ddab90d886537bc4b17b,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-19 01:45:44,1,0x464d47e357e52d40a9f5ddab90d886537bc4b17b,45486,0,2022-05-19 01:49:54,85.0,4.166667,7879,1104
73,0xe85ca190c6c4967c4cde7f358e158268d97c2ff3,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-19 02:01:30,1,0xe85ca190c6c4967c4cde7f358e158268d97c2ff3,45486,0,2022-05-19 02:04:24,92.0,2.900000,7879,1104
...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,0x53a58d08fdce1bb95839859f083ab5ca33812881,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-19 06:11:32,1,0x53a58d08fdce1bb95839859f083ab5ca33812881,45486,0,2022-05-19 06:12:32,199.0,1.000000,7879,1104
179,0x60bb2686a464a366e7c257fd218e84348d5b5882,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-19 06:12:32,1,0x60bb2686a464a366e7c257fd218e84348d5b5882,45486,0,2022-05-19 06:13:16,200.0,0.733333,7879,1104
180,0x234867eaed8b60f5cc928c9b2ab5392806aa31cb,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-19 06:13:16,1,0x234867eaed8b60f5cc928c9b2ab5392806aa31cb,45486,0,2022-05-19 06:14:56,201.0,1.666667,7879,1104
181,0x982ddd238b6c56a97607d7ce37e2e48b0b0d7e2b,0xdb46d1dc155634fbc732f92e853b10b288ad5a1d,0xa22cb465,2022-05-19 06:14:56,1,0x982ddd238b6c56a97607d7ce37e2e48b0b0d7e2b,45486,0,2022-05-19 06:15:00,202.0,0.066667,7879,1104


In [25]:
result_df[result_df['cluster_id'] == 691]['node_addr'].nunique()

103

In [24]:

result_df[result_df['cluster_id'] == 1104]['node_addr'].nunique()


101